In [9]:
from google import genai
from google.genai import types
import pathlib
import httpx
import os
import json

from Gen_flashCard_SubAgent.ConceptPlanner import Agent_ConceptPlanner
from Gen_flashCard_SubAgent.CardWriter import Agent_CardWriter
from Gen_flashCard_SubAgent.QualityValidator import Agent_QualityValidator


In [10]:
def Generate_flashCard(user_profile, lecture_material, target_card_count=15):
    
    # [Step 1] Concept Planner: 무엇을 낼지 계획
    print(f"\n[Step 1] 핵심 개념 추출 및 출제 계획 수립 중... (목표 카드 수: {target_card_count})")
    # 강의 자료에서 Profile의 의도(심층 이해, 범위 등)에 맞는 핵심 개념 추출
    concept_plan = Agent_ConceptPlanner(
        source_text=lecture_material, 
        profile=user_profile,
        target_count=target_card_count
    )
    print(">> 출제 계획 수립 완료.")

    # [Step 2] Generation & Validation Loop (Max 3)
    current_feedback = None # 초기 피드백은 없음
    generated_json = None   # 결과물을 담을 변수
    
    MAX_RETRIES = 3
    
    for attempt in range(MAX_RETRIES):
        print(f"\n[Step 2-{attempt+1}] 플래시 카드 생성 및 검증 시도 (Attempt {attempt+1}/{MAX_RETRIES})")
        
        # 2-1. Card Writer: 계획과 피드백을 반영하여 JSON 생성 (Self-Schema Awareness)
        # *Writer는 이미 정의된 'flash_card' 스키마를 알고 있다고 가정함
        print("  - 카드 생성 중...")
        generated_json_str = Agent_CardWriter(
            source_text=lecture_material,    # 강의 자료 원문 참조
            plan=concept_plan,               # 출제 계획
            feedback=current_feedback,       # 이전 턴의 피드백 (없으면 null)
            prior_content = generated_json, # 이전 턴의 플래시 카드 (없으면 null)
            num_cards=target_card_count,     # 목표 개수
            profile=user_profile             # 스타일 가이드(언어, 시나리오 등)
        )

        generated_json = json.loads(generated_json_str)
        print(f"  - {len(generated_json['flash_cards'])}개의 카드 생성 완료.")

        # 2-2. Quality Validator: 품질 검증
        print("  - 품질 검증 중...")
        validation_result_str = Agent_QualityValidator(
            target_content=generated_json, # 방금 만든 카드 내용
            source =lecture_material,  # 원본 강의 자료 (Fact Check용)
            guideline=user_profile,        # 스타일 준수 여부 확인
            required_count=target_card_count
        )

        validation_result = json.loads(validation_result_str)
        
        # 2-3. Decision Making
        if validation_result['is_valid'] == True:
            # 검증 통과 시 루프 즉시 종료 및 반환
            print(">> 품질 검증 통과! 최종 결과물을 반환합니다.")
            return generated_json
        else:
            # 검증 실패 시 피드백을 업데이트하고 루프 재실행
            # 예: "3번 카드의 설명이 강의 자료와 다릅니다. 수식 유도를 더 포함하세요."
            print(f"  !! 품질 검증 미통과: {validation_result['feedback_message']}")
            current_feedback = validation_result['feedback_message']
            print("  - 피드백을 반영하여 재생성을 시도합니다.")
            continue

    # [Fallback] 최대 횟수 초과 시, 마지막으로 생성된 결과라도 반환 (혹은 에러 처리)
    # (일반적으로는 마지막 피드백을 반영하려고 노력한 결과를 반환함)
    print(f"\n!! 최대 재시도 횟수({MAX_RETRIES})에 도달했습니다. 현재까지의 결과물을 반환합니다.")

    # 딕셔너리 형태로 변환하여 반환
    return generated_json

In [11]:
# 가짜 데이터
user_profile = {
    "learning_goal": {
        "focus_areas": [
            "삼성전자 실적",
            "삼성전자 전망"
        ],
        "target_depth": "Deep Understanding",
        "question_modality": "Conceptual"
    },
    "user_status": {
        "proficiency_level": "Intermediate",
        "weakness_focus": True
    },
    "interaction_style": {
        "language_preference": "Korean_with_Korean_Terms",
        "scenario_based": True
    },
    "feedback_preference": {
        "strictness": "Strict",
        "explanation_depth": "Detailed_with_Examples"
    },
    "scope_boundary": "Lecture_Material_Only"
}
lecture_material = "/Users/jhkim/Desktop/LectureTestGenerator/SamSung(260109).pdf"
target_card_count = 10

In [12]:
response = Generate_flashCard(user_profile, lecture_material, target_card_count)


[Step 1] 핵심 개념 추출 및 출제 계획 수립 중... (목표 카드 수: 10)
>> 출제 계획 수립 완료.

[Step 2-1] 플래시 카드 생성 및 검증 시도 (Attempt 1/3)
  - 카드 생성 중...
  - 10개의 카드 생성 완료.
  - 품질 검증 중...
  !! 품질 검증 미통과: [{'id': 2, 'message': "Scenario mismatch. 사용자의 'scenario_based' 가이드라인을 준수하여, 단순 질문 대신 특정 상황이나 분석가의 역할을 가정한 시나리오 형태로 문항을 재구성하세요."}, {'id': 4, 'message': 'Scenario mismatch. 사용자는 시나리오 기반의 학습을 선호합니다. VD/가전 사업부의 적자 원인을 조사하는 회의 상황 등을 설정하여 시나리오를 추가하세요.'}, {'id': 5, 'message': "Scenario mismatch. 'scenario_based' 규칙에 따라 미래 시장 전망을 예측하는 투자 전략 시나리오 등을 도입하여 질문을 수정하세요."}, {'id': 6, 'message': "Scenario & Language violation. 시나리오 형식을 도입하고, 'ASP', 'Shipment', 'B/G'와 같은 영문 용어를 '평균판매단가', '출하량', '비트 성장률' 등 한글 용어로 수정하여 'Korean_with_Korean_Terms' 규칙을 준수하세요."}, {'id': 8, 'message': "Scenario & Language violation. 시나리오 형식을 도입하세요. 또한 'Valuation'을 '가치평가'로 변경하고, BPS와 PBR 역시 한글 용어(주당순자산가치, 주가순자산비율)를 우선적으로 사용하세요."}, {'id': 9, 'message': "Scenario & Language violation. 시나리오 형식을 도입하고, 'Bit Growth'를 한글 용어인 '출하량 증가율' 또는 '비트 성장률'로 변경하세요."}, {'id

In [13]:
print(response)

{'flash_cards': [{'id': 1, 'front_content': "[시나리오] 삼성전자의 2025년 4분기 잠정 실적을 분석한 결과, 전 분기 대비 매출액과 영업이익이 각각 8.0%, 67.3% 증가했습니다. 이러한 성장세에도 불구하고 보고서가 '실적이 전망치 대비 감소했다'고 평가한 구체적인 지표 값은 얼마인가요?", 'back_content': '매출액 93조원, 영업이익 20.3조원입니다. 이는 전 분기 대비 증가한 수치이나, 당초 시장의 기대치와 이전 전망치에는 미치지 못한 규모입니다.', 'category_tag': '2025년 4분기 실적 총평'}, {'id': 2, 'front_content': '[시나리오] 당신은 삼성전자 반도체 부문의 실적 개선 원인을 분석하는 애널리스트입니다. 2025년 4분기, 전사 이익 증가액(8조 원)을 상회하는 10조 원의 이익 개선을 달성한 DS 부문의 구체적인 원인 세 가지를 보고서 내용에 기반하여 제시하시오.', 'back_content': '1. DRAM 수요 증가에 따른 가격 상승, 2. SSD 수요 증가에 따른 가격 상승, 3. 고대역폭 메모리(HBM) 물량 증가가 주요 원인입니다.', 'category_tag': 'DS 부문 실적 동인'}, {'id': 3, 'front_content': '[시나리오] 메모리 반도체 가격의 상승은 DS 부문에는 이익 증가 요인이 되지만, MX(모바일) 사업부의 수익성에는 어떤 상반된 영향을 미치는지 설명하시오.', 'back_content': '메모리 가격 상승은 스마트폰 제조 원가 구조를 악화시켜 MX 사업부의 수익성을 저해하는 주요 원인이 됩니다. 실제로 25년 4분기 MX 부문은 예상치 대비 크게 부진했습니다.', 'category_tag': '사업부별 수익성 상관관계'}, {'id': 4, 'front_content': '[시나리오] 경영진 회의에서 VD 및 가전 사업부의 2025년 4분기 대규모 영업적자(-8,180억 원)에 대한 원인 규명을 요청받